In [51]:
SET NOCOUNT ON;
SET ANSI_WARNINGS OFF;


DECLARE @dinicio DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), GETDATE(), 121) + ' 00:00:00');

DECLARE @dfim  DATETIME = CONVERT(DATETIME, CONVERT(VARCHAR(10), GETDATE(), 121) + ' 23:59:00');


DECLARE @ligacoes TABLE (
	DATA DATE
	,LOGIN VARCHAR(30)
	,TotalAgentCalls INT
	,TotalAgentTime INT
	,PreviewTime INT
	,PreviewQt INT
	,ActiveTime INT
	,WrapTime INT
	)

INSERT INTO @ligacoes
SELECT cast(DATEADD(HOUR, - 3, BeginTimePeriodDt) AS DATE) AS DATA
	,user_id AS LOGIN
	,Sum([TotalAgentCalls])
	,Sum([TotalAgentTime])
	,Sum([PreviewTime])
	,Sum(CASE 
			WHEN [PreviewTime] > 0
				THEN 1
			ELSE 0
			END) AS PreviewQt
	,Sum([ActiveTime])
	,Sum([WrapTime])
FROM summary_epro..[MediaAgentSummary] WITH (NOLOCK)
WHERE DATEADD(HOUR, - 3, BeginTimePeriodDt) BETWEEN @dinicio
		AND @dfim
GROUP BY cast(DATEADD(HOUR, - 3, BeginTimePeriodDt) AS DATE)
	,user_id

-- Criação da tabela temporária
DECLARE @work TABLE (
	DATA DATE
	,ID_DISCADOR_OPERADOR VARCHAR(30)
	,LOGIN_DISCADOR_OPERADOR VARCHAR(100)
	,NOME_DISCADOR_OPERADOR VARCHAR(100)
	,LOGIN_DISCADOR_SUPERVISOR VARCHAR(100)
	,LOGIN TIME
	,LOGOUT TIME
	,QTD_LOGIN INT
	,QTD_LOGOUT INT
	,TEMPO_LOGADO INT
	,TEMPO_DISPONIVEL INT
	,TEMPO_IDLE INT
    , CAMPANHA VARCHAR(20)
    , ID_CAMPANHA INT
    , ID_NOVO INT
    
	);

-- Inserção de dados na tabela temporária
INSERT INTO @work
SELECT CAST(DATEADD(HOUR, - 3, t1.LoginDt) AS DATE) AS DATA
	,t1.WorkGroup_Id AS ID_DISCADOR_OPERADOR
	,t1.User_Id AS LOGIN_DISCADOR_OPERADOR
	,USR.User_F_Name + ' ' + USR.User_L_Name AS NOME_DISCADOR_OPERADOR
	,[WK].[Workgroup_Name] AS LOGIN_DISCADOR_SUPERVISOR
	    
    ,MIN(DATEADD(HOUR, - 3, t1.LoginDt)) AS LOGIN
	,MAX(DATEADD(HOUR, - 3, ISNULL(t1.LogoutDt, @dfim))) AS LOGOUT
	,COUNT(t1.LoginDt) AS QTD_LOGIN
	,COUNT(t1.LogoutDt) AS QTD_LOGOUT
	,SUM(CASE 
			WHEN t1.LogoutDt IS NOT NULL
				THEN t1.LoginTime
			ELSE DATEDIFF(SECOND, t1.LoginDt, ISNULL(t1.LogoutDt, @dfim))
			END) AS TEMPO_LOGADO
	,SUM(t1.GapTime) AS TEMPO_DISPONIVEL
	,SUM(t1.IdleTime) AS TEMPO_IDLE
    ,(SER.Service_c) AS CAMPANHA  
    ,(SER.Service_Id) AS ID_CAMPANHA
    ,DENSE_RANK() OVER (
			PARTITION BY t1.User_Id  ORDER BY SER.Service_c
			) AS ID_NOVO
FROM summary_epro..[AgentSignInSummary] t1 WITH (NOLOCK)
INNER JOIN config_epro..[Workgroup] wk WITH (NOLOCK) ON t1.Workgroup_Id = wk.Workgroup_Id
INNER JOIN [config_epro]..[Service_Group] AS [SERG] WITH (NOLOCK) ON [SERG].[User_Id] = t1.User_Id
INNER  JOIN [config_epro]..[Service] AS SER  WITH (NOLOCK) ON SER.Service_Id = SERG.Service_Id
JOIN [config_epro]..[Users] AS [USR] WITH (NOLOCK) ON [SERG].[User_Id] = [USR].[User_Id]
WHERE DATEADD(HOUR, - 3, t1.LoginDt) BETWEEN @dinicio
		AND @dfim
GROUP BY CAST(DATEADD(HOUR, - 3, t1.LoginDt) AS DATE)
	,t1.User_Id
	,t1.WorkGroup_Id
	,wk.Workgroup_Name
	,USR.User_F_Name + ' ' + USR.User_L_Name 
    ,(SER.Service_c) 
    ,(SER.Service_Id)

-- Consulta dos dados na tabela temporária
SELECT w.DATA
    ,w.ID_CAMPANHA
    ,w.CAMPANHA
	,w.ID_DISCADOR_OPERADOR
	,w.LOGIN_DISCADOR_OPERADOR
	,w.NOME_DISCADOR_OPERADOR
	,w.LOGIN_DISCADOR_SUPERVISOR
	,w.LOGIN
	,w.LOGOUT
	,w.QTD_LOGIN
	,w.QTD_LOGOUT
	,isnull(l.TotalAgentCalls, 0) AS ATENDIDAS
	,w.TEMPO_LOGADO
	,
	-- Calcula o total de segundos
	DATEDIFF(SECOND, w.LOGIN, w.LOGOUT) - IIF(DATEDIFF(SECOND, w.LOGIN, w.LOGOUT) - w.TEMPO_LOGADO > 0, w.TEMPO_LOGADO, 0) AS TEMPO_DESLOGADO
	,ISNULL(l.ActiveTime, 0) + ISNULL(l.WrapTime, 0) + w.TEMPO_IDLE AS TEMPO_DISPONIVEL
	,
	-- Tempo de pausa em segundos
	w.TEMPO_LOGADO - (ISNULL(l.ActiveTime, 0) + ISNULL(l.WrapTime, 0) + w.TEMPO_IDLE) AS TEMPO_PAUSA
	,
	-- Tempo falado em segundos
	ISNULL(l.ActiveTime, 0) AS TEMPO_FALADO
	,
	-- Tempo de pós-atendimento em segundos
	ISNULL(l.WrapTime, 0) AS TEMPO_POS
	,
	-- Calcula o tempo deslogado em segundos
	w.TEMPO_IDLE,
    ISNULL(l.ActiveTime, 0)  /  isnull(l.TotalAgentCalls, 0)  AS TMA ,
    ISNULL(l.WrapTime, 0)  /  isnull(l.TotalAgentCalls, 0)  AS TMP ,
    w.TEMPO_IDLE /  isnull(l.TotalAgentCalls, 0)  AS TME ,
   ( ISNULL(l.ActiveTime, 0)  /  isnull(l.TotalAgentCalls, 0)) + (ISNULL(l.WrapTime, 0)  /  isnull(l.TotalAgentCalls, 0) ) AS TMO
  
FROM @work w
LEFT JOIN @ligacoes l ON (
		l.DATA = w.DATA
		AND l.LOGIN = w.LOGIN_DISCADOR_OPERADOR
		)

WHERE w.ID_NOVO = 1
AND w.LOGIN_DISCADOR_OPERADOR != 'rstaspectagent'
and w.DATA = cast(@dinicio as date)


Commands completed successfully.

Total execution time: 00:00:31.750

DATA,ID_NOVO,ID_CAMPANHA,CAMPANHA,ID_DISCADOR_OPERADOR,LOGIN_DISCADOR_OPERADOR,NOME_DISCADOR_OPERADOR,LOGIN_DISCADOR_SUPERVISOR,LOGIN,LOGOUT,QTD_LOGIN,QTD_LOGOUT,ATENDIDAS,TEMPO_LOGADO,TEMPO_DESLOGADO,TEMPO_DISPONIVEL,TEMPO_PAUSA,TEMPO_FALADO,TEMPO_POS,TEMPO_IDLE,TMA,TMP,TME,TMO
2024-07-29,1,9,MF_TIM_TELECOB,9,TIMADRIANACAMPOS,ADRIANA OLIVEIRA CAMPOS,TIM_COB_SUP_JULIANA_VON_PAUMGARTTEN_DE_ARAUJO,08:13:54.7690000,14:19:20.8450000,1,1,225,21926,21926,19008,2918,10390,1742,6876,46,7,30,53
2024-07-29,1,9,MF_TIM_TELECOB,10,TIMADRIANACLE,ADRIANA CLEMENTE FERREIRA,TIM_COB_SUP_MIRIAN_MOTA_TORRES,14:23:11.6380000,20:43:58.1470000,1,1,171,22847,22847,19540,3307,9421,988,9131,55,5,53,60
2024-07-29,1,10,MF_TIM_LIVE_TC,10,TIMALAN5059,ALAN CASTRO DA SILVA,TIM_COB_SUP_MIRIAN_MOTA_TORRES,14:23:16.9980000,21:06:26.2600000,1,1,153,24189,1,20982,3207,5216,1291,14475,34,8,94,42
2024-07-29,1,15,TIM_CALLBACK_MANUAL,5,TIMALCILENE7858,AUCILENE MENDES DE SOUSA,TIM_COB_SUP_HELDER_RODRIGUES_MACIEL,14:26:57.9830000,21:06:26.2910000,1,1,197,23968,1,23968,0,8974,1261,13733,45,6,69,51
2024-07-29,1,10,MF_TIM_LIVE_TC,4,TIMALESSANDRA441,ALESSANDRA DOS SANTOS MARTINS,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,14:29:32.3920000,21:06:26.2550000,1,1,155,23814,23814,21426,2388,4170,1124,16132,26,7,104,33
2024-07-29,1,10,MF_TIM_LIVE_TC,4,TIMALEXIA2281,ALEXIA MARIA DE SOUSA LEITE,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,07:45:39.7290000,14:26:16.4600000,1,1,153,24037,24037,19726,4311,4650,1465,13611,30,9,88,39
2024-07-29,1,9,MF_TIM_TELECOB,7,TIMALISSON6134,ALISSON JOSE COSTA BASTOS,TIM_COB_SUP_KARYANNE_SUELLEN_LIMA_DE_OLIVEIRA,08:08:54.5170000,14:20:21.2410000,1,1,207,22287,22287,18855,3432,8239,2885,7731,39,13,37,52
2024-07-29,1,9,MF_TIM_TELECOB,4,TIMANA1689,ANA LIVIA MENDES DE SOUSA,TIM_COB_SUP_PAULA_VANESSA_SOUZA_DA_SILVA,14:44:52.8770000,21:06:26.3190000,1,1,198,22893,1,20603,2290,7164,1962,11477,36,9,57,45
2024-07-29,1,9,MF_TIM_TELECOB,10,TIMANA2027,ANA CAROLINE DA COSTA TORRES,TIM_COB_SUP_MIRIAN_MOTA_TORRES,14:23:04.7630000,20:41:13.3930000,1,1,185,22689,22689,19297,3392,7484,1642,10171,40,8,54,48
2024-07-29,1,9,MF_TIM_TELECOB,5,TIMANA2945,ANA JULIA SILVA DOS SANTOS DIAS,TIM_COB_SUP_HELDER_RODRIGUES_MACIEL,14:29:31.0030000,21:06:26.3060000,1,1,157,23815,23815,20946,2869,6090,1688,13168,38,10,83,48


In [39]:
SELECT * FROM [config_epro]..[Service] 

Commands completed successfully.

Total execution time: 00:00:00.012

Service_Id,Priority,Schedule_Id,DispPlan_Id,RouteAccess_Id,ServiceType_Id,Available_f,Service_short,Service_c,Application_Id,CCID,BackUpCCID,Music_Id,OutgoingMask,EverBeenActivated,Msg_Id,LanguageId,CallDataDefId,CatalogId,PresentationIndicator,CpnAllowOverride,CallingNumber,SerializeCallDataFlag,ServiceNameMessageId,ServiceNameLanguageId,AffinityTimeout,StaticPriority,ServiceLevel,CallToAgentRatio,GoalQueueTime,CARatioCalls,CARatioAgents,CallingPartyName,RecordingScheduleId,Preemptive,RecordingDuplex,RecordTariffTone,RecordTariffToneDuplex,MonitorTariffTone,MonitorTariffToneDuplex,RecordTariffToneInterval,MonitorTariffToneInterval,AuditableAlertForAcceptCall,MultiLine,MultiLineNoAnswerTimeout,AutoRecordConsultation,PGAllowed,PG_Id,PGSchedule_Id,AutoRecordHeldCalls,AcceptMsgId,TimeZone_Id,ResetStatsByTimeZone
0,0,0,0,0,0,0,Nenhum Serviço,Nenhum Serviço,0,0,0,0,0,0,0,1,NULL,NULL,0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,1,NULL,1,NULL,NULL,NULL,0,-1,18,0
1,1,1,2,12,1,1,ACDRST,ACD_RST,1,NULL,NULL,NULL,7,0,0,10,4,NULL,NULL,0,NULL,1,0,10,NULL,5,80,2,NULL,10,5,NULL,NULL,1,0,2,-1,-1,NULL,NULL,NULL,0,1,20,1,0,NULL,NULL,1,NULL,17,0
2,1,1,2,11,4,1,ALMRST,Servico_ALM_RST,1,NULL,NULL,NULL,7,0,444,10,17,NULL,NULL,0,NULL,1,444,10,NULL,5,90,2,NULL,2,1,NULL,-1,1,0,-1,-1,-1,-1,NULL,NULL,0,0,5,1,1,NULL,NULL,1,NULL,17,0
3,NULL,1,2,12,3,1,IVRRST,IVR_RST,1,NULL,NULL,NULL,NULL,0,0,10,4,NULL,NULL,0,NULL,1,0,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,0,0,NULL,0,0,NULL,NULL,0,NULL,17,0
4,0,2,3,11,4,1,trei,treinamento,2,NULL,NULL,0,7,0,0,1,16,NULL,NULL,0,NULL,1,0,1,NULL,10,90,2,NULL,20,10,NULL,2,1,0,-1,-1,-1,-1,NULL,NULL,0,0,5,0,1,NULL,NULL,0,NULL,17,0
5,0,3,4,12,4,1,TIM_1,TIM_1,3,NULL,NULL,0,7,0,444,10,17,NULL,NULL,NULL,NULL,1,444,10,NULL,5,90,NULL,NULL,14,7,NULL,3,1,0,-1,-1,-1,-1,NULL,NULL,0,0,5,1,1,NULL,NULL,0,-1,17,0
6,0,3,4,12,4,1,TIM_2,TIM_2,3,NULL,NULL,0,7,0,444,10,17,NULL,NULL,NULL,NULL,1,444,10,NULL,5,80,NULL,NULL,60,30,NULL,3,1,0,-1,-1,-1,-1,NULL,NULL,0,0,5,1,1,NULL,NULL,0,-1,17,0
7,0,3,6,12,4,1,TIM_3,TIM_3,3,NULL,NULL,NULL,7,0,444,10,17,NULL,NULL,0,NULL,1,444,10,NULL,5,90,2,NULL,32,16,NULL,3,1,0,-1,-1,-1,-1,NULL,NULL,0,0,5,1,1,NULL,NULL,0,NULL,17,0
9,1,3,4,12,1,1,TELCOB,MF_TIM_TELECOB,3,NULL,NULL,NULL,7,0,0,10,4,NULL,NULL,0,NULL,1,0,10,NULL,0,80,2,NULL,10,5,NULL,NULL,1,0,2,-1,-1,NULL,NULL,NULL,0,1,20,1,0,NULL,NULL,0,NULL,17,0
10,1,3,4,12,1,1,LIVETC,MF_TIM_LIVE_TC,3,NULL,NULL,NULL,7,0,0,10,4,NULL,NULL,0,NULL,1,0,10,NULL,0,80,2,NULL,10,5,NULL,NULL,1,0,2,-1,-1,NULL,NULL,NULL,0,1,20,1,0,NULL,NULL,0,NULL,17,0


In [20]:
SELECT 
    DE.LoginDt, 
    DE.User_Id, 
    DE.LogoutDt,
    SER.Service_c AS CAMPANHA,
    SER.Service_Id
FROM 
    detail_epro..AgentLoginLogout DE
INNER JOIN 
    [config_epro]..[Service] AS SER WITH (NOLOCK) 
    ON SER.Service_Id = DE.Service_Id
WHERE 
    CAST(DE.LoginDt AS DATE) = '2024-07-26' 
        AND DE.User_Id = 'TIMANANAYRA';


Commands completed successfully.

Total execution time: 00:00:00.019

LoginDt,User_Id,LogoutDt,CAMPANHAA,Service_Id
2024-07-26 11:00:48.452,TIMANANAYRA,2024-07-26 15:30:25.777,TIM_4,21
2024-07-26 10:43:11.751,TIMANANAYRA,2024-07-26 17:25:04.259,Nenhum Serviço,0
2024-07-26 11:00:01.884,TIMANANAYRA,2024-07-26 17:25:04.259,MF_TIM_TELECOB,9
2024-07-26 15:32:57.726,TIMANANAYRA,2024-07-26 17:25:04.259,TIM_4,21


In [28]:
SELECT * FROM [config_epro]..[Service_Group]  WITH (NOLOCK)

Commands completed successfully.

Total execution time: 00:00:00.019

Service_Id,User_Id,Agent_Order,Skill_Weight,ReviewEmails,CDAssignedFlag,AutoRecording
1,admvictor,2,0,0,0,0
1,llusr,4,NULL,NULL,0,0
1,rstaspectagent,1,0,0,1,0
1,rstaspectinstall,3,NULL,NULL,0,0
2,admcesaraugusto,5,0,0,1,0
2,llusr,2,0,0,1,0
2,rstaspectagent,1,0,0,1,0
2,tiluana,6,NULL,NULL,1,0
2,tiodovaldo,3,0,0,1,0
5,llusr,286,NULL,NULL,1,0
